# «Функции и работа с данными»

# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('ml-latest-small/ratings.csv')
movies.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
def classify(rating):
    if rating <= 2:
        return 'low'
    elif rating <= 4:
        return 'medium'
    elif 4 < rating <= 5:
        return 'high'
    else:
        return 'n/a'

In [9]:
movies['class'] = movies['rating'].apply(classify)
movies['class'].value_counts()

medium    63801
high      22818
low       13385
Name: class, dtype: int64

# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Результат классификации запишите в отдельный столбец region.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [10]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [11]:
keywords = pd.read_csv('Python_11_pandas_functions_groupby/keywords.csv')
keywords.head() 

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [17]:
def classify(keyword):
    for key, value in geo_data.items():
        for city in value:
            if city in keyword:
                return key
    
    return 'undefined'

In [24]:
classify('сахалинпетербур')

'Дальний Восток'

In [28]:
keywords['region'] = keywords['keyword'].apply(classify)
keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

# Задание 3
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
 1. для каждой строки пройдите по всем годам списка years
 2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
 3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [45]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
movie_ratings = pd.merge(ratings, movies, on='movieId')
movie_ratings

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama
...,...,...,...,...,...,...
99999,664,64997,2.5,1343761859,War of the Worlds (2005),Action|Sci-Fi
100000,664,72380,3.5,1344435977,"Box, The (2009)",Drama|Horror|Mystery|Sci-Fi|Thriller
100001,665,129,3.0,995232528,Pie in the Sky (1996),Comedy|Romance
100002,665,4736,1.0,1010197684,Summer Catch (2001),Comedy|Drama|Romance


In [55]:
years = range(1950, 2011)

In [50]:
def production_year(movie_title):
    for y in years:
        if str(y) in movie_title:
            return y
        
    return 1900

In [73]:
movie_ratings['year'] = movie_ratings['title'].apply(production_year)
movie_ratings.groupby('year').agg(['mean'])['rating'].sort_values('mean', ascending=False)

,mean
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
...,...
2005,3.448434
2003,3.444777
1996,3.426600
